In [1]:
import argparse
from os.path import exists
# import confusionmatrix
import os
import warnings
from demv import DEMV
from utils import *
# import getdataset
from fairlearn.reductions import ExponentiatedGradient, BoundedGroupLoss, ZeroOneLoss, GridSearch
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import sys
sys.path.append('../')
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [41]:
ML_METHODS = {
    'logreg': LogisticRegression(),
    'svm': SVC()
}
FAIRNESS_METHODS = ['demv', 'eg', 'grid']
METRICS = ['disparate_impact', 'statistical_parity', 'equalized_odds', 'accuracy', 'hmean']
metrics = {
    'stat_par': [],
    'eq_odds': [],
    'zero_one_loss': [],
    'disp_imp': [],
    'acc': [],
    'hmean': []
}

In [42]:
data = pd.read_csv('data/cmc.data', names=['wife_age', 'wife_edu', 'hus_edu',
                   'num_child', 'wife_religion', 'wife_work', 'hus_occ', 'living', 'media', 'contr_use'])
label = 'contr_use'
sensitive_features = ['wife_religion', 'wife_work']
unpriv_group = {'wife_religion': 1, 'wife_work': 1}
positive_label= 2

In [43]:
ris = pd.DataFrame()

In [44]:
for m in ML_METHODS:
  if m == 'logreg':
    classifier = LogisticRegression()
  if m == 'svm':
    classifier = SVC()
  else:
    classifier = GradientBoostingClassifier()
  model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', classifier)
  ])
  for f in FAIRNESS_METHODS:
    model = deepcopy(model)
    data = data.copy()
    if f == 'no_one':
      fair_model, metrics = cross_val(model, data, label, unpriv_group, sensitive_features, positive_label)
    if f == 'demv':
      demv = DEMV(round_level=1)
      fair_model, metrics = cross_val(
          model, data, label, unpriv_group, sensitive_features, positive_label, debiaser=demv)
    if f == 'eg':
      constr = BoundedGroupLoss(ZeroOneLoss(),upper_bound=0.1)
      eg = ExponentiatedGradient(model, constraints=constr, sample_weight_name="classifier__sample_weight")
      fair_model, metrics = cross_val(
          eg, data, label, unpriv_group, sensitive_features, positive_label, exp=True)
    else:
      constr = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
      grid = GridSearch(
          model, constraints=constr, sample_weight_name="classifier__sample_weight")
      fair_model, metrics = cross_val(
          grid, data, label, unpriv_group, sensitive_features, positive_label, exp=True)
    df_metrics = pd.DataFrame(metrics)
    df_metrics = df_metrics.explode(list(df_metrics.columns))
    df_metrics['model'] = m
    df_metrics['fairness_method'] = f
    ris = ris.append(df_metrics)

In [57]:
ris.groupby(['fairness_method']).agg(np.mean).sort_values('hmean',ascending=False)

,stat_par,eq_odds,zero_one_loss,disp_imp,acc,hmean
fairness_method,,,,,,
eg,-0.099442,0.171602,0.323072,0.603158,0.546481,0.668711
grid,-0.124971,0.244910,0.246549,0.464498,0.437537,0.492534
demv,-0.128669,0.237804,0.272517,0.453334,0.441607,0.488290
